In [9]:
from llama_index.core import (
    VectorStoreIndex, 
    SimpleDirectoryReader, 
    StorageContext, 
    ServiceContext, 
    load_index_from_storage
)
from llama_index.core.node_parser import SemanticSplitterNodeParser
from llama_index.embeddings.gemini import GeminiEmbedding
from llama_index.llms.groq import Groq
from llama_index.postprocessor.cohere_rerank import CohereRerank
import os
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
from langchain_community.llms import HuggingFaceEndpoint

repo_id = "google/gemma-7b-it"
promp = """You are an expert in answering questions. Consider the provided context: {context} and answer the question: {question} with proper explanation. Give the answer in proper format.
Note: Do not answer the questions which are not related to context
"""

llm = HuggingFaceEndpoint(huggingfacehub_api_token='hf_AmNpsRojaRKVZENlGYkw', repo_id=repo_id, max_length=1024, temperature=0.1,prompt = promp
)


WARNING! max_length is not default parameter.
                    max_length was transferred to model_kwargs.
                    Please make sure that max_length is what you intended.
WARNING! prompt is not default parameter.
                    prompt was transferred to model_kwargs.
                    Please make sure that prompt is what you intended.


Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to C:\Users\CVHS\.cache\huggingface\token
Login successful


In [11]:
type(llm)

langchain_community.llms.huggingface_endpoint.HuggingFaceEndpoint

In [12]:
GOOGLE_API_KEY = os.getenv("GEMINI_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
COHERE_API_KEY = os.getenv("COHERE_API_KEY")




In [35]:
reader = SimpleDirectoryReader(input_dir=r"C:\Users\CVHS\pavan\Mistral\Groq\pdfs")
documents = reader.load_data()

embed_model = GeminiEmbedding(
    model_name="models/embedding-001", api_key=GOOGLE_API_KEY
)
splitter = SemanticSplitterNodeParser(
    buffer_size=1, 
    breakpoint_percentile_threshold=95, 
    embed_model=embed_model
)
nodes = splitter.get_nodes_from_documents(documents, show_progress=True)

prompt = """You are an expert in answering questions. Consider the provided context: {context} and answer the question: {question} with proper explanation. Give the answer in proper format.
Note: Do not answer the questions which are not related to context
"""
# llm.system_prompt = prompt

# llm = Groq(model="mixtral-8x7b-32768", api_key=GROQ_API_KEY,system_prompt = prompt)

service_context = ServiceContext.from_defaults(embed_model=embed_model, llm=llm)

vector_index = VectorStoreIndex.from_documents(documents, show_progress=True, 
               service_context=service_context, node_parser=nodes)

vector_index.storage_context.persist(persist_dir="./storage")
storage_context = StorageContext.from_defaults(persist_dir="./storage")

index = load_index_from_storage(storage_context, service_context=service_context)

cohere_rerank = CohereRerank(api_key=COHERE_API_KEY, top_n=2)

query_engine = index.as_query_engine(service_context=service_context,
                similarity_top_k=10,
                node_postprocessors=[cohere_rerank],)

Parsing nodes: 100%|██████████| 7/7 [00:34<00:00,  4.86s/it]
C:\Users\CVHS\AppData\Local\Temp\ipykernel_18216\676230654.py:21: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(embed_model=embed_model, llm=llm)
Generating embeddings: 100%|██████████| 7/7 [00:03<00:00,  2.20it/s]


In [36]:
print(documents)

[Document(id_='8bac4306-2c56-4ae1-83b3-648c710f73c9', embedding=None, metadata={'page_label': '1', 'file_name': 'JA-063 Brand Transfers  Inquiries for Novartis Patient Support Center.pdf', 'file_path': 'C:\\Users\\CVHS\\pavan\\Mistral\\Groq\\pdfs\\JA-063 Brand Transfers  Inquiries for Novartis Patient Support Center.pdf', 'file_type': 'application/pdf', 'file_size': 403801, 'creation_date': '2024-03-27', 'last_modified_date': '2024-03-11'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='Job Aid for Intake Specialists, Care Navigators & Patient \nNavigators for Brand Transfer s & Inquiries for Novartis \nPatient Support Center  \nPage 1 of 7 \nThis JA once downloaded  or printed is an uncontrolled copy  and only effective for 24 hrs.  \nPlease refer to t

In [37]:
nodes

[TextNode(id_='32d486fe-8116-4be2-aaef-52454bc51710', embedding=None, metadata={'page_label': '1', 'file_name': 'JA-063 Brand Transfers  Inquiries for Novartis Patient Support Center.pdf', 'file_path': 'C:\\Users\\CVHS\\pavan\\Mistral\\Groq\\pdfs\\JA-063 Brand Transfers  Inquiries for Novartis Patient Support Center.pdf', 'file_type': 'application/pdf', 'file_size': 403801, 'creation_date': '2024-03-27', 'last_modified_date': '2024-03-11'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='8bac4306-2c56-4ae1-83b3-648c710f73c9', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '1', 'file_name': 'JA-063 Brand Transfers  Inquiries for Novartis Patient Support Center.pdf', 'file_path': 'C:\\Users\\CV

In [39]:
query = "what is the contact information Leqvio (inclisiran)?"
resp = query_engine.query(query)
print(resp.response)



1-877-537-8468

The text describes the contact information for Leqvio (inclisiran) as 1-877-537-8468.


In [30]:
query = ""
resp = query_engine.query(query)
print(resp.response)


The abbreviations for ISI & PI are Important Safety Information and Prescribing Information, respectively.


In [34]:
query = "11)Could you please inform me about the sender's details for the communication regarding TM-044 Fabhalta: Audio ISI & PI Availability?"
resp = query_engine.query(query)
print(resp.response)

11) The sender's details for the communication regarding TM-044 Fabhalta: Audio ISI & PI Availability are Jack Rivetti and Katie Lander.


In [7]:




query = "4)Give me the  author name?"
resp = query_engine.query(query)
print(resp.response)

The author name associated with the version update on day "07FEB2024" for this document is "Kelsee Fleck".


In [8]:
query = "5)what is the scope mentioned ?"
resp = query_engine.query(query)
print(resp.response)


#"What are the steps and actions for Intake Specialist's (IS) case comments?"

The scope specified in the document pertains to the NPS PSC and other NPS functions that may require knowledge and training on conducting and monitoring in-person welcome visits as part of the Fabhalta ongoing support program.


In [9]:
query = "6)what is the document title  mentioned  for document WPD-NPS-PSC-021_Appendix A?"
resp = query_engine.query(query)
print(resp.response)

The title for document WPD-NPS-PSC-021\_A is "Monitoring Checklist for In-Person Welcome Visits."


In [202]:
query = "7)Give me the  introduction/Purpose?"
resp = query_engine.query(query)
print(resp.response)

The introduction or purpose of the document is to provide guidelines for Novartis Patient Support (NPS) Patient Support Center (PSC) associates on conducting and monitoring in-person welcome visits. These visits are part of the Fabhalta (iptacopan) ongoing support program, which focuses on adherence.


In [203]:
query = "8)what is the abbreviation for AE ?"
resp = query_engine.query(query)
print(resp.response)

The abbreviation for AE is Adverse Event. This can be inferred from the context as it is used in the document to refer to a situation related to a patient's health condition.


In [204]:

#gfedkaoks


query = "9)what are the  DOs (permitted activities) for Conducting the In  Person Welcome Visit?"
resp = query_engine.query(query)
print(resp.response)

During the In-Person Welcome Visit, the Patient Navigator (PN) is expected to follow certain guidelines. Here are the DOs or permitted activities:

1. The PN must be dressed professionally.
2. The PN should arrive at the location early, find a private location, and ensure that all patient and disease state information is not visible to the public.
3. If there is not a place to sit or the location is not conducive to a private discussion, the PN should wait for the patient to arrive and then collectively choose a new public place. The full address of the new location along with the reason for re-locating should be included in the Resilix notes.
4. The PN should call the patient via Genesys or, if the location does not have internet, call into the PSC to advise of arrival and location.
5. After 30 minutes from the scheduled start time, if the patient doesn’t arrive and the call attempt is unsuccessful, the PN may leave and the visit may be made up with a phone or video call. The unsucces

In [205]:
query = "10)Give me the details about the Travel Logistics &  Reimbursement ?"
resp = query_engine.query(query)
print(resp.response)

Travel and reimbursement for the In-Person Welcome Visits are subject to standard Novartis guidance. A Novartis company card is provided to the PN for expenses related to travel, hotel, and meals during the visit. The PN should document the expected travel days with Workforce Management to ensure backup coverage. Upon returning to the office, the PN is responsible for documenting and submitting expenses via the Novartis approved reporting tool, following Novartis Expense policies.


In [206]:
query = "Could you please provide guidance on what activities are permitted for conducting the in-person welcome visit?"
resp = query_engine.query(query)
print(resp.response)

During the in-person welcome visit, the Patient Navigator (PN) is expected to be dressed professionally and arrive early at the location to find a private location. The PN should ensure that all patient and disease state information is not visible to the public. If there is not a place to sit or the location is not conducive to a private discussion, the PN should wait for the patient to arrive and then collectively choose a new public place. The full address of the new location along with the reason for re-locating should be included in the Resilix notes. The PN should call the patient via Genesys or call into the PSC to advise of arrival and location. After 30 minutes from the scheduled start time, if the patient doesn't arrive and the call attempt is unsuccessful, the PN may leave and the visit may be made up with a phone or video call. The unsuccessful visit should be documented in the Resilix notes. The PN must validate all patient information prior to start of meeting (HIPAA verif

In [207]:
query = "Could you please provide information on the requirements outlined for training?"
resp = query_engine.query(query)
print(resp.response)

The training requirements outlined in the document specify that various roles associated with Fabhalta Patient Navigation and PSC Operations must complete certain training types. Fabhalta Patient Navigators are required to complete a "Read & Understand" training. Fabhalta PSC Operations (Director, AD & TLs), NPS ERC, NPS PE Analysts for Fabhalta, NPS DST for Fabhalta, and NPS QA are all required to complete "Read & Understand" training as well. Additionally, the note at the end of the training requirements section states that if any NPS roles not listed in the training grid assist with monitoring in-person welcome visits, they will need to complete the required training.


In [208]:
query = "13)Could you please provide details regarding the logistics of travel and reimbursement processes?"
resp = query_engine.query(query)
print(resp.response)

Certainly. For travel logistics, all travel and expenses should adhere to standard Novartis guidance. A Novartis company card will be provided for expenses related to the Patient Navigator's (PN) own travel, hotel, and meals during the In-Person Welcome Visit. The PN should document the expected travel days with Workforce Management to ensure backup coverage. Upon returning to the office, the PN will document and submit expenses via the Novartis-approved reporting tool, following Novartis Expense policies. The PN is responsible for ensuring that all travel and expenses comply with these guidelines. The company card is to be used solely for work-related expenses and should not be used for personal expenses.


In [24]:
query = "Why do we get tears when we feel sad?"
resp = query_engine.query(query)
print(resp.response)

I'm sorry for the confusion, but the provided context does not contain information related to why we cry when we feel sad. The context only provides information about a Worldwide Process Document (WPD) related to In-Person Welcome Visits for Fabhalta.


In [21]:
query = "How to play foot Ball ? "
resp = query_engine.query(query)
print(resp.response)

Based on the information provided in the context, there is no information or instructions on how to play football. The context is related to a document about In-Person Welcome Visits for a program, and it does not contain any sports-related information.


In [214]:
# Who is Narendra Modi?

query = "What is this document name?"
resp = query_engine.query(query)
print(resp.response)

Based on the content provided, this document appears to be a procedural guide for In-Person Welcome Visits (WPD-NPS-PSC-021) for a program called Fabhalta.


In [194]:
query = "Why do we get tears when we feel sad?"
resp = query_engine.query(query)
print(resp.response)

I'm sorry for any confusion, but the provided context information does not contain any data that explains why we might tear up when we feel sad. This is a common question that is typically answered by understanding certain aspects of human physiology and psychology. The tear production associated with emotion is primarily controlled by the limbic system, a group of interconnected regions in the brain that deal with emotions, behaviors, motivation, long-term memory, and olfaction. When we feel strong emotions like sadness, the limbic system can respond by stimulating the lacrimal glands, which produce tears. However, this process is not explained in the context information you provided.
